In [11]:
from spark_utils import get_spark_session
import pyspark.sql.functions as F

spark = get_spark_session()
# run transform_latest.py if this don't exist
test_data = spark.read.parquet('data_transformed/amex-default-prediction/test_data_latest')
train_data = spark.read.parquet('data_transformed/amex-default-prediction/train_data_latest')
# run format_data.py if these don't exist
train_labels = spark.read.parquet('data/amex-default-prediction/train_labels')
sample_submission = spark.read.parquet('data/amex-default-prediction/sample_submission')

In [12]:
%%time
from format_data import CATEGORICAL_VARIABLES
from encoder import CategoricalToIntegerEncoders, HolidayFeatures

encs = CategoricalToIntegerEncoders(
    columns=CATEGORICAL_VARIABLES
).fit(train_data)

years = [row['S_2'].year for row in train_data.select('S_2').distinct().collect()]
years += [row['S_2'].year for row in test_data.select('S_2').distinct().collect()]
years = sorted(list(set(years)))
print(years)
years = [min(years) + i for i in range(-1, max(years) - min(years) + 2)]
print(years)

holiday = HolidayFeatures(
    column='S_2',
    years=years,
)

[2018, 2019]
[2017, 2018, 2019, 2020]
CPU times: user 53.2 ms, sys: 120 µs, total: 53.3 ms
Wall time: 6.65 s


In [13]:
%%time
train_pdf = train_data.join(train_labels, on='customer_ID', how='inner')
train_pdf = encs.transform(spark=spark, df=train_pdf)
train_pdf = holiday.transform(spark=spark, df=train_pdf)
train_pdf = train_pdf.toPandas()

CPU times: user 465 ms, sys: 336 ms, total: 801 ms
Wall time: 8.9 s


In [14]:
%%time
test_pdf = test_data
test_pdf = encs.transform(spark=spark, df=test_pdf)
test_pdf = holiday.transform(spark=spark, df=test_pdf)
test_pdf = test_pdf.toPandas()

CPU times: user 815 ms, sys: 3.29 s, total: 4.1 s
Wall time: 11.9 s


In [15]:
from format_data import TARGET_VARIABLE, DATE_VARIABLES, ID_VARIABLES

feature_columns = [
    c for c in train_pdf.columns 
    if c not in [TARGET_VARIABLE,] + ID_VARIABLES + list(DATE_VARIABLES.keys())
]
', '.join(feature_columns)

'P_2, D_39, B_1, B_2, R_1, S_3, D_41, B_3, D_42, D_43, D_44, B_4, D_45, B_5, R_2, D_46, D_47, D_48, D_49, B_6, B_7, B_8, D_50, D_51, B_9, R_3, D_52, P_3, B_10, D_53, S_5, B_11, S_6, D_54, R_4, S_7, B_12, S_8, D_55, D_56, B_13, R_5, D_58, S_9, B_14, D_59, D_60, D_61, B_15, S_11, D_62, D_65, B_16, B_17, B_18, B_19, B_20, S_12, R_6, S_13, B_21, D_69, B_22, D_70, D_71, D_72, S_15, B_23, D_73, P_4, D_74, D_75, D_76, B_24, R_7, D_77, B_25, B_26, D_78, D_79, R_8, R_9, S_16, D_80, R_10, R_11, B_27, D_81, D_82, S_17, R_12, B_28, R_13, D_83, R_14, R_15, D_84, R_16, B_29, S_18, D_86, D_87, R_17, R_18, D_88, B_31, S_19, R_19, B_32, S_20, R_20, R_21, B_33, D_89, R_22, R_23, D_91, D_92, D_93, D_94, R_24, R_25, D_96, S_22, S_23, S_24, S_25, S_26, D_102, D_103, D_104, D_105, D_106, D_107, B_36, B_37, R_26, R_27, D_108, D_109, D_110, D_111, B_39, D_112, B_40, S_27, D_113, D_115, D_118, D_119, D_121, D_122, D_123, D_124, D_125, D_127, D_128, D_129, B_41, B_42, D_130, D_131, D_132, D_133, R_28, D_134, D_

In [16]:
X = train_pdf[feature_columns]
y = train_pdf[TARGET_VARIABLE]
print(y.unique())

# this is a new bit
postive_label_multiplicative_factor = 1./20.

# Note that the negative class has been subsampled for this dataset at 5%, and thus receives a 20x weighting in the scoring metric.
w = y.apply(lambda x: postive_label_multiplicative_factor*20. if x == 1. else 1.)
print(w.unique())

[1. 0.]
[1.]


In [17]:
X_test = test_pdf[feature_columns]

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid, w_train, w_valid = train_test_split(X, y, w)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape, w_train.shape, w_valid.shape

((344184, 190), (114729, 190), (344184,), (114729,), (344184,), (114729,))

In [19]:
%%time
import os
from tempfile import TemporaryDirectory
from format_data import PREDICTION_VARIABLE
import pandas as pd
from evaluation import evaluate
from lightgbm import LGBMClassifier
import mlflow


mlflow.lightgbm.autolog()
experiment_id = mlflow.get_experiment_by_name('use_latest+holiday.ipynb').experiment_id
with mlflow.start_run(experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    print(f'run_id: {run_id}')

    # weight related feature
    mlflow.log_param('postive_label_multiplicative_factor', postive_label_multiplicative_factor)

    m = LGBMClassifier().fit(
        X=X_train, y=y_train, sample_weight=w_train,
        categorical_feature=encs.columns_encoded,
    )
    # these are auto logged
    score_train = m.score(X=X_train, y=y_train, sample_weight=w_train)
    score_valid = m.score(X=X_valid, y=y_valid, sample_weight=w_valid)
    # these are not, so we'll need to log them manually
    score_amex_train = evaluate(X_train, y_train, m=m)
    score_amex_valid = evaluate(X_valid, y_valid, m=m)
    mlflow.log_metric('score_amex_train', score_amex_train)
    mlflow.log_metric('score_amex_valid', score_amex_valid)

    pred_df = pd.DataFrame({
        PREDICTION_VARIABLE: m.predict_proba(X_test)[:, 1],
        'customer_ID': test_pdf['customer_ID'],
    })
    pred_and_sample_joined_counts = (
        spark
        .createDataFrame(pred_df)
        .join(sample_submission, on='customer_ID', how='inner')
        .count()
    )
    assert pred_and_sample_joined_counts == len(pred_df), \
        f'''These should be identical:
        sample_submission has {sample_submission.count()} rows,
        pred_and_sample_joined_counts is {pred_and_sample_joined_counts},
        pred_df has {len(pred_df)} rows
        '''
    with TemporaryDirectory() as p:
        p = os.path.join(p, 'submission.csv')
        pred_df.to_csv(p, header=True, index=False)
        mlflow.log_artifact(local_path=p)


run_id: 8cc58c1faa8a41ed8ec683f1a0fda6c9


/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
2022/06/17 15:15:23 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.8/dist-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for mo

CPU times: user 1min 41s, sys: 10.3 s, total: 1min 51s
Wall time: 32.4 s
